In [5]:
import numpy as np

pose2d = np.load("./data/fs_tas/features/2dvpd/men_olympic_short_program_2010_01_00011475_00015700.npy")
pose3d = np.load("./data/fs_tas/features/pose3d/men_olympic_short_program_2010_01_00011475_00015700.npy")

In [6]:
print(pose2d.shape, pose3d.shape)

(4225, 52) (4225, 54)


In [44]:
pose3d[2000]

array([ 0.        ,  0.        ,  0.        ,  0.1       , -0.1       ,
        0.        ,  0.16145063, -0.27814506, -0.02155036,  0.07966797,
       -0.29427808, -0.09153423, -0.1       , -0.1       ,  0.        ,
       -0.51444057, -0.26169251,  0.31070377,  0.24323118,  0.99225348,
       -0.16727259,  0.07784026,  0.1255988 , -0.4155766 ,  0.32875326,
        0.3369973 , -0.01390487,  0.        ,  0.5       ,  0.        ,
       -0.05461721, -0.32891146,  0.37189825,  0.4316351 , -0.10616841,
        0.1711811 , -0.45344965, -0.22956074, -0.46443459, -0.16494427,
       -0.58486245,  0.55479526, -0.41521079, -0.34873852,  0.08194098,
       -0.1300648 , -0.28276005,  0.07838274,  0.42528242, -0.20167466,
       -0.06687867,  0.07917083,  0.84152812, -0.07275678])

In [73]:
data_reshaped = pose3d.reshape(-1, 18, 3)[2000:2200,:,:]

In [74]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def plot_3d(pose_3d):
    all_poses = pose_3d
    poses = []

    for frame in range(len(all_poses)):
        vals = all_poses[frame]
        joint_positions = vals[:17].copy()
        euler_angles = vals[17].copy()
        
        rot = R.from_euler('xyz', -euler_angles, degrees=True)
        # rot = R.from_rotvec(-euler_angles)
        joint_positions = rot.apply(joint_positions)
        
        poses.append(joint_positions)

    return np.array(poses)

In [75]:
poses_global = plot_3d(data_reshaped)
poses_global[0]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00114089e-01, -9.98728557e-02,  1.60685506e-03],
       [ 1.62097187e-01, -2.77968959e-01, -1.87925050e-02],
       [ 8.13715395e-02, -2.94301186e-01, -8.99476106e-02],
       [-9.98641786e-02, -1.00126797e-01, -1.33052664e-03],
       [-5.18621490e-01, -2.61921712e-01,  3.03475989e-01],
       [ 2.44421889e-01,  9.92332572e-01, -1.65053012e-01],
       [ 8.37791116e-02,  1.25130932e-01, -4.14561684e-01],
       [ 3.28500688e-01,  3.37395183e-01, -9.54059946e-03],
       [-6.24774815e-04,  4.99999132e-01, -6.90821048e-04],
       [-5.96629623e-02, -3.28473291e-01,  3.71510065e-01],
       [ 4.29206447e-01, -1.05386836e-01,  1.77648541e-01],
       [-4.46291615e-01, -2.30769175e-01, -4.70726654e-01],
       [-1.72344718e-01, -5.84314599e-01,  5.53120445e-01],
       [-4.15933507e-01, -3.49153415e-01,  7.63157337e-02],
       [-1.30848680e-01, -2.82817860e-01,  7.68546389e-02],
       [ 4.26470563e-01, -2.01225495e-01

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def get_canonical_orientation(X, torso_forward_vec, spine_up_vec,
                              interp_start=45, interp_range=30):
    X_zm = X - np.mean(X, axis=0).flatten()
    _, _, V = np.linalg.svd(X_zm)
    torso_forward_vec = -V[2, :] if V[2, :].dot(torso_forward_vec) < 0 else V[2, :]
    spine_up_vec = -V[0, :] if V[0, :].dot(spine_up_vec) < 0 else V[0, :]
    torso_pitch = np.degrees(np.arcsin(torso_forward_vec[2]))
    if torso_pitch > interp_start:
        if torso_pitch < interp_start + interp_range:
            theta = (torso_pitch - interp_start) / interp_range
            return theta * -spine_up_vec + (1. - theta) * torso_forward_vec
        else:
            return -spine_up_vec
    elif torso_pitch < -interp_start:
        if torso_pitch > -interp_start - interp_range:
            theta = (-torso_pitch - interp_start) / interp_range
            return theta * spine_up_vec + (1. - theta) * torso_forward_vec
        else:
            return spine_up_vec
    else:
        return torso_forward_vec

def h36m_global_to_local(pose_3d_global):
    """
    Convert H36M global joint positions to local with canonical orientation alignment.
    
    H36M 17-joint skeleton:
    0: Hip, 1: RHip, 2: RKnee, 3: RAnkle, 4: LHip, 5: LKnee, 6: LAnkle  
    7: Spine, 8: Thorax, 9: Neck, 10: Head, 11: LShoulder, 12: LElbow, 13: LWrist
    14: RShoulder, 15: RElbow, 16: RWrist
    
    Args:
        pose_3d_global: (nframe, 17, 3) array of global joint positions
    
    Returns:
        pose_3d_local: (nframe, 18, 3) array where:
                      - First row (index 0) contains euler angles (rx, ry, rz)
                      - Remaining 17 rows contain local joint positions
    """
    nframes, njoints, _ = pose_3d_global.shape
    pose_3d_local = np.zeros((nframes, 18, 3))
    
    Z_UNIT = np.array([0, 0, 1])
    HIP, NECK, L_HIP, R_HIP, L_SHOULDER, R_SHOULDER = 0, 9, 4, 1, 11, 14
    
    for frame in range(nframes):
        xyz = pose_3d_global[frame].copy()
        
        root_position = xyz[HIP].copy()
        xyz = xyz - root_position
        
        key_joints = xyz[[HIP, L_SHOULDER, R_SHOULDER, NECK, L_HIP, R_HIP], :]
        torso_forward_vec = np.cross(xyz[L_HIP, :] - xyz[HIP, :], xyz[R_HIP, :] - xyz[HIP, :])
        spine_up_vec = xyz[NECK, :] - xyz[HIP, :]
        
        forward_vec = get_canonical_orientation(key_joints, torso_forward_vec, spine_up_vec)
        forward_vec[2] = 0
        forward_vec = forward_vec / np.linalg.norm(forward_vec)
        lateral_vec = np.cross(Z_UNIT, forward_vec)
        rot_mat = np.array([lateral_vec, forward_vec, Z_UNIT]).T
        xyz = xyz.dot(rot_mat)
        
        theta = np.degrees(np.arccos(np.clip(lateral_vec[0], -1, 1)))
        if lateral_vec[1] < 0:
            theta = -theta
        
        euler_angles = np.array([0, 0, theta])
        
        pose_3d_local[frame, 17, :] = euler_angles
        pose_3d_local[frame, :17, :] = xyz
    
    return pose_3d_local

In [66]:
poses_local = h36m_global_to_local(pose_3d_global=poses_global)
poses_local[0]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.41214014e-01,  7.48466445e-03,  1.60685506e-03],
       [ 3.06282224e-01,  9.86581943e-02, -1.87925050e-02],
       [ 2.57100833e-01,  1.64723031e-01, -8.99476106e-02],
       [-7.46971883e-03,  1.41217679e-01, -1.33052664e-03],
       [-2.11125220e-01,  5.41292319e-01,  3.03475989e-01],
       [-4.80737476e-01, -9.01863334e-01, -1.65053012e-01],
       [-2.12007345e-02, -1.49087956e-01, -4.14561684e-01],
       [ 1.92086649e-02, -4.70509552e-01, -9.54059946e-03],
       [-3.34360738e-01, -3.71755860e-01, -6.90821048e-04],
       [ 1.74942038e-01,  2.84340738e-01,  3.71510065e-01],
       [ 3.89855267e-01, -2.08176439e-01,  1.77648541e-01],
       [-1.78090093e-01,  4.69802657e-01, -4.70726654e-01],
       [ 2.61916577e-01,  5.50023598e-01,  5.53120445e-01],
       [-7.64370008e-02,  5.37648747e-01,  7.63157337e-02],
       [ 9.14667744e-02,  2.97894525e-01,  7.68546389e-02],
       [ 4.51819161e-01, -1.35012174e-01

In [56]:
# data_reshaped[0] == poses_local[0]
for j in range(18):
    for i in range(3):
        print(data_reshaped[0][j][i], poses_local[0][j][i])

0.0 0.0
0.0 0.0
0.0 0.0
0.1 0.08077789455737432
-0.1 -0.08426823823933811
0.0 0.07983605560719967
0.16145062948361583 0.24215935057600577
-0.2781450603924403 -0.17515222952392107
-0.021550357462093616 0.12073087305872049
0.07966796832965257 0.26393196224026255
-0.29427807724616345 -0.17738611044226626
-0.09153423387788763 0.014112938292717174
-0.1 0.11495300922634645
-0.1 0.044540052968931446
0.0 -0.06929638772210492
-0.5144405742717794 0.3794877675531537
-0.26169250904630825 0.5088708000761782
0.3107037739014052 -0.16342720695147345
0.24323118284870893 -1.0317346536090215
0.9922534770172997 -0.08312153278111875
-0.1672725879200816 0.017969556554744505
0.07784025798322973 -0.18366900687337442
0.12559880481510344 -0.3321907382104551
-0.41557660458160395 -0.2246176510668612
0.3287532632116563 -0.3875674800396097
0.3369973012020388 -0.15506150610229888
-0.013904871919862984 0.21814355890997902
0.0 -0.4893272594593019
0.5 0.09932046317601666
0.0 -0.02634916971273684
-0.05461720989646907 0.

In [215]:
def estimate_alignment_euler(L_hip, R_hip, neck, order="xyz"):
    # 1. Estimate axes
x_axis = R_hip - L_hip                 # right
y_axis = neck - 0.5 * (L_hip + R_hip)  # forward
z_axis = np.cross(x_axis, y_axis)     # up (perpendicular to both)

# 2. Re-orthogonalize y_axis
y_axis = np.cross(z_axis, x_axis)

# 3. Normalize
x_axis /= np.linalg.norm(x_axis)
y_axis /= np.linalg.norm(y_axis)
z_axis /= np.linalg.norm(z_axis)

# 4. Stack into rotation matrix
R_align = np.stack([x_axis, y_axis, z_axis], axis=1)  # shape (3, 3)

# 5. Check determinant
det = np.linalg.det(R_align)
if det < 0:
    print("Warning: flipped frame — flipping Z to fix")
    R_align[:, 2] *= -1  # Flip Z to restore right-handedness
    print(R_align)
    return R.from_matrix(R_align).as_euler(order, degrees=False)

In [80]:
HIP, NECK, L_HIP, R_HIP, L_SHOULDER, R_SHOULDER = 0, 9, 4, 1, 11, 14
print(
    estimate_alignment_euler(
        poses_global[0][L_HIP, :] - poses_global[0][HIP, :],
        poses_global[0][R_HIP, :] - poses_global[0][HIP, :],
        poses_global[0][NECK, :] - poses_global[0][HIP, :],
    )
)

[[ 0.99989134 -0.00124955  0.01468864]
 [ 0.00126971  0.99999826 -0.00136314]
 [ 0.01468691 -0.00138164 -0.99989119]]


ValueError: Non-positive determinant (left-handed or null coordinate frame) in rotation matrix 0: [[ 0.99989134 -0.00124955  0.01468864]
 [ 0.00126971  0.99999826 -0.00136314]
 [ 0.01468691 -0.00138164 -0.99989119]].